In [1]:
import pandas as pd
import numpy as np

In [2]:
DB_PATH = "Simulation/FSN_Data_5k.db"

In [3]:
from NetEmbs.utils.IO.db_connection import upload_data, upload_journal_entries
from NetEmbs.DataProcessing import *

In [4]:
data = upload_data(DB_PATH, limit=1000)

In [5]:
d = data_preprocessing(data)

In [6]:
data.head(18).to_dict()

{'ID': {'0': 1,
  '1': 1,
  '2': 1,
  '3': 2,
  '4': 2,
  '5': 2,
  '6': 3,
  '7': 3,
  '8': 3,
  '9': 4,
  '10': 4,
  '11': 4,
  '12': 5,
  '13': 5,
  '14': 5,
  '15': 6,
  '16': 6,
  '17': 6},
 'Name': {'0': 'Revenue_11',
  '1': 'Tax_11',
  '2': 'TradeReceivables_11',
  '3': 'Revenue_11',
  '4': 'Tax_11',
  '5': 'TradeReceivables_11',
  '6': 'Revenue_2',
  '7': 'Tax_2',
  '8': 'TradeReceivables_2',
  '9': 'Revenue_4',
  '10': 'Tax_4',
  '11': 'TradeReceivables_4',
  '12': 'Revenue_8',
  '13': 'Tax_8',
  '14': 'TradeReceivables_8',
  '15': 'Revenue_0',
  '16': 'Tax_0',
  '17': 'TradeReceivables_0'},
 'FA_Name': {'0': 'Revenue',
  '1': 'Tax',
  '2': 'TradeReceivables',
  '3': 'Revenue',
  '4': 'Tax',
  '5': 'TradeReceivables',
  '6': 'Revenue',
  '7': 'Tax',
  '8': 'TradeReceivables',
  '9': 'Revenue',
  '10': 'Tax',
  '11': 'TradeReceivables',
  '12': 'Revenue',
  '13': 'Tax',
  '14': 'TradeReceivables',
  '15': 'Revenue',
  '16': 'Tax',
  '17': 'TradeReceivables'},
 'Value': {'0': 0.

In [7]:
from NetEmbs import *
GLOBAL_FSN = FSN()
GLOBAL_FSN.build(d, left_title="FA_Name")

In [35]:
fin_walk = finWalk(GLOBAL_FSN)

In [43]:
fin_walk.walk(7)

[7, 17, 146, 158, 47, 118, 60, 130, 91, 110]

In [33]:
GLOBAL_FSN.out_edges('TradePayables', data=True)

OutEdgeDataView([('TradePayables', 7, {'type': 'CREDIT', 'weight': 0.9964662946731169}), ('TradePayables', 16, {'type': 'CREDIT', 'weight': 1.0}), ('TradePayables', 17, {'type': 'CREDIT', 'weight': 0.9968529075486792}), ('TradePayables', 28, {'type': 'CREDIT', 'weight': 1.0}), ('TradePayables', 37, {'type': 'CREDIT', 'weight': 1.0}), ('TradePayables', 55, {'type': 'CREDIT', 'weight': 1.0}), ('TradePayables', 67, {'type': 'CREDIT', 'weight': 1.0}), ('TradePayables', 75, {'type': 'CREDIT', 'weight': 1.0}), ('TradePayables', 97, {'type': 'CREDIT', 'weight': 0.9877493568254404}), ('TradePayables', 146, {'type': 'CREDIT', 'weight': 1.0})])